# 1. Classification: 
Classification is to identify which category a new observation belongs to, on the basis of a training dataset. There are five datasets. For each dataset, we provide the training dataset, training label, and test dataset. Please use the training dataset and training label to build your classifier and predict the test label. A class label is represented by an integer. For example, in the 1st dataset, there are 4 classes where 1 represents the 1st class, 2 represents the 2nd class, etc. Note that, there exist some missing values in some of the dataset (a missing entry is filled by 1.00000000000000e+99), please fill the missing values before perform your classification algorithm.

TrainData 1 contains 3312 features with 150 samples. Testdata1 contains 3312 features with 53 samples. There are 5 classes in this dataset.

TrainData 2 contains 9182 features with 100 samples. Testdata2 contains 9182 features with 74 samples. There are 11 classes in this dataset.

TrainData 3 contains 13  features with 6300 samples. Testdata3 contains 13 features with 2693 samples. There are 9 classes in this dataset.

TrainData 4 contains 112 features with 2547 samples. Testdata4 contains 112 features with 1092 samples. There are 9 classes in this dataset.

TrainData 5 contains 11 features with 1119 samples. Testdata5 contains 11 features with 480 samples. There are 6 classes in this dataset.

In [29]:
# Importing Libraries
import numpy as np
import pandas as pd
import random
from random import seed
from random import randrange
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

In [22]:
# Importing the datasets
traindata1 = pd.read_csv('input/TrainData1.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
traindata2 = pd.read_csv('input/TrainData2.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
traindata3 = pd.read_csv('input/TrainData3.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
traindata4 = pd.read_csv('input/TrainData4.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
traindata5 = pd.read_csv('input/TrainData5.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
train_data = [traindata1, traindata2, traindata3, traindata4, traindata5]

testdata1 = pd.read_csv('input/TestData1.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
testdata2 = pd.read_csv('input/TestData2.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
testdata3 = pd.read_csv('input/TestData3.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
testdata4 = pd.read_csv('input/TestData4.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
testdata5 = pd.read_csv('input/TestData5.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
test_data = [testdata1, testdata2, testdata3, testdata4, testdata5]

trainlabel1 = pd.read_csv('input/TrainLabel1.txt', sep='\t', header=None)
trainlabel2 = pd.read_csv('input/TrainLabel2.txt', sep='\t', header=None)
trainlabel3 = pd.read_csv('input/TrainLabel3.txt', sep='\t', header=None)
trainlabel4 = pd.read_csv('input/TrainLabel4.txt', sep='\t', header=None)
trainlabel5 = pd.read_csv('input/TrainLabel5.txt', sep='\t', header=None)
train_label = [trainlabel1, trainlabel2, trainlabel3, trainlabel4, trainlabel5]

In [23]:
# count the numbers of coloumns where the value = 1.00000000000000e+99
def count_na(data, type):
    # print("Missing values in the " + type + " data:")
    for i in range(len(data)):
        missing_val_count = data[i].isnull().sum().sum()
        # print( type + " dataset " + str(i+1) + ": " + str(missing_val_count))

count_na(train_data, "train")
count_na(test_data, "test")

In [24]:
# Now we need to replace the missing values in the dataset using the mean of each column
def replace_na(data):
    for i in range(len(data)):
        for column in data[i].columns:   
                data[i][column] = data[i][column].fillna(data[i][column].mean())
         
replace_na(train_data)

# Replacing test dataset 1 missing values
for column in testdata1.columns:
    testdata1[column] = testdata1[column].fillna(testdata1[column].mean())

# Now checking if any remaining missing vals left
# count_na(train_data, "train")
# count_na(test_data, "test")


In [13]:
# For all the train data, remove the columns with low variance
# def remove_low_variance(data):
#     for i in range(len(data)):
#         selector = VarianceThreshold(threshold=.8 * (1 - .8))
#         data[i] = selector.fit_transform(data[i])

# remove_low_variance(train_data)

In [14]:
# remove correlated features
# def remove_correlation(data, threshold):
#     for i in range(len(data)):
#         data[i] = pd.DataFrame(data[i])
#         col_corr = set()  # Set of all the names of correlated columns
#         corr_matrix = data[i].corr()
#         for k in range(len(corr_matrix.columns)):
#             for j in range(k):
#                 if abs(corr_matrix.iloc[k, j]) > threshold:
#                     colname = corr_matrix.columns[j]
#                     col_corr.add(colname)

#         data[i] = data[i].drop(data[i].columns[list(col_corr)], axis=1)

# remove_correlation(train_data, 0.8)

In [25]:
# Splitting the train data into training and testing
X_train = []
X_test = []
y_train = []
y_test = []

def split_data(data, label):
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state = 21)
    return X_train, X_test, y_train, y_test

for i in range(len(train_data)):
    X_train_i, X_test_i, y_train_i, y_test_i = split_data(train_data[i], train_label[i])
    X_train.append(X_train_i)
    X_test.append(X_test_i)
    y_train.append(y_train_i)
    y_test.append(y_test_i)
    # print("Train dataset " + str(i+1) + ": " + str(len(X_train_i)))
    # print("Test dataset " + str(i+1) + ": " + str(len(X_test_i)))

In [30]:
# Naive Bayes Classifier
class NaiveBayesClassifier():
    '''
    Bayes Theorem form
    P(y|X) = P(X|y) * P(y) / P(X)
    '''
    def calc_prior(self, features, target):
        '''
        prior probability P(y)
        calculate prior probabilities
        '''
        self.prior = (features.groupby(target).apply(lambda x: len(x)) / self.rows).to_numpy()

        return self.prior
    
    def calc_statistics(self, features, target):
        '''
        calculate mean, variance for each column and convert to numpy array
        ''' 
        self.mean = features.groupby(target).apply(np.mean).to_numpy()
        self.var = features.groupby(target).apply(np.var).to_numpy()
              
        return self.mean, self.var
    
    def gaussian_density(self, class_idx, x):     
        '''
        calculate probability from gaussian density function (normally distributed)
        we will assume that probability of specific target value given specific class is normally distributed 
        
        probability density function derived from wikipedia:
        (1/√2pi*σ) * exp((-1/2)*((x-μ)^2)/(2*σ²)), where μ is mean, σ² is variance, σ is quare root of variance (standard deviation)
        '''
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp((-1/2)*((x-mean)**2) / (2 * var))
#         numerator = np.exp(-((x-mean)**2 / (2 * var)))
        denominator = np.sqrt(2 * np.pi * var)
        prob = numerator / denominator
        return prob
    
    def calc_posterior(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for i in range(self.count):
            prior = np.log(self.prior[i]) ## use the log to make it more numerically stable
            conditional = np.sum(np.log(self.gaussian_density(i, x))) # use the log to make it more numerically stable
            posterior = prior + conditional
            posteriors.append(posterior)
        # return class with highest posterior probability
        return self.classes[np.argmax(posteriors)]
     

    def fit(self, features, target):
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        
        self.calc_statistics(features, target)
        self.calc_prior(features, target)
        
    def predict(self, features):
        preds = [self.calc_posterior(f) for f in features.to_numpy()]
        return preds

    def accuracy(self, y_test, y_pred):
        accuracy = np.sum(y_test == y_pred) / len(y_test)
        return accuracy

    def visualize(self, y_true, y_pred, target):
        
        tr = pd.DataFrame(data=y_true, columns=[target])
        pr = pd.DataFrame(data=y_pred, columns=[target])
        
        
        fig, ax = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(15,6))
        
        sns.countplot(x=target, data=tr, ax=ax[0], palette='viridis', alpha=0.7, hue=target, dodge=False)
        sns.countplot(x=target, data=pr, ax=ax[1], palette='viridis', alpha=0.7, hue=target, dodge=False)
        

        fig.suptitle('True vs Predicted Comparison', fontsize=20)

        ax[0].tick_params(labelsize=12)
        ax[1].tick_params(labelsize=12)
        ax[0].set_title("True values", fontsize=18)
        ax[1].set_title("Predicted values", fontsize=18)
        plt.show()

In [34]:
# function to get accuracy
def get_accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

nb = NaiveBayesClassifier()



nb.fit(X_train[2], y_train[2])
predictions = nb.predict(X_test[2].values)
print(nb.accuracy(y_test[2], predictions))

ValueError: Grouper for '<class 'pandas.core.frame.DataFrame'>' not 1-dimensional